In [141]:
from newspaper import fulltext, Article
from datetime import date, timedelta
import requests
import pandas as pd
from dateutil import parser
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pytz
from selectorlib import Extractor
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException, UnexpectedAlertPresentException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from custom_scripts import *
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [142]:
%timeit
def fetch_articles(company):
    ChromeOptions = webdriver.ChromeOptions()
    ChromeOptions.add_argument('--incognito')
    ChromeOptions.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=ChromeOptions)
    time.sleep(2)
    print('navigating to newslookup..')
    news_look = "https://www.newslookup.com"
    time.sleep(1)
    driver.get(news_look)
    time.sleep(3)
    
    driver.find_element_by_css_selector('#lookup').send_keys(company)
    
    #search button
    driver.find_element_by_css_selector('#form-group > div > span > button').click()
    
    #time period
    driver.find_element_by_css_selector('#timeperiod').click()
    
    #past 24 hours--------------------------------------------------
    driver.find_element_by_css_selector('#tp_12').click()
    
    #scroll down
    for i in range(10000):
        try:
            print(f'{i}')
            time.sleep(1.5)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1.5)
            driver.find_element_by_css_selector('#more-btn').click()
        except ElementNotInteractableException:
            print('no more pages')
            break
        except (TimeoutException, ElementClickInterceptedException) as e:
            continue
        except NoSuchElementException:
            break
    
    pano = []

    r = driver.page_source
    e = Extractor.from_yaml_string("""
    card:
        css: 'div#results'
        xpath: null
        type: Text
        children:
            title:
                css: 'a.title:nth-of-type(n+4)'
                xpath: null
                multiple: true
                type: Link
            date:
                css: span.stime
                xpath: null
                multiple: true
                type: Text

     """)
    dat = e.extract(r)

    inf = list(zip(dat['card']['title'],dat['card']['date'][1:]))
    for entry in inf:
        pano.append(entry)
        
    return pano

In [143]:
test_list = fetch_articles('apple')

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/brendanferris/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
navigating to newslookup..
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
no more pages


In [144]:
from multiprocessing.dummy import Pool as ThreadPool

In [189]:
def getTxt(lst):
    url, posttime = lst
    article = Article(url)
    article.download()
    try:
        article.parse()
        text = article.text
        auth = article.authors
        source = lst
        summ = article.summary
        titl = article.title
        dic = {'time':posttime,'source':url, 'author':auth, 'fulltext':text, 'summary':summ, 'title':titl}

        return dic
    except Exception as e:
        return None #we can remove null values from results list.

    

In [190]:
%%time
    
pool = ThreadPool(15)
# open the urls in their own threads
# and return the results
raw_output = pool.map(getTxt, test_list)
results = [record for record in raw_output if record] #only return a list of truthy values. 

# close the pool and wait for the work to finish 
pool.close() 
pool.join()    

CPU times: user 19min 36s, sys: 10min 42s, total: 30min 18s
Wall time: 11min 18s


In [191]:
%%timeit
pd.DataFrame(results)

100 loops, best of 3: 3.46 ms per loop


In [193]:
v = pd.DataFrame(results)

In [198]:
v.drop_duplicates(subset=['fulltext'])

,time,source,author,fulltext,summary,title
0,2021-01-05 22:18 UTC,https://www.cnbc.com/2021/01/05/apple-proxy-20...,[Kif Leswing],"Tim Cook, chief executive officer of Apple, sp...",,Apple adds new section about antitrust risk to...
1,2021-01-06 00:04 UTC,https://www.malaymail.com/news/money/2021/01/0...,[],Living up to Apple’s stated values regarding s...,,"Apple ties exec bonuses to social, environment..."
2,2021-01-05 23:35 UTC,https://www.ibtimes.com/apple-ties-exec-bonuse...,[],Apple is making environmental and social value...,,"Apple Ties Exec Bonuses To Social, Environment..."
3,2021-01-05 23:24 UTC,https://www.reuters.com/article/apple-compensa...,[Stephen Nellis],SAN FRANCISCO (Reuters) -Apple Inc said in its...,,Apple will modify executive bonuses based on e...
7,2021-01-05 16:56 UTC,https://www.businessinsider.in/stock-market/ne...,[],Warren Buffett's Berkshire Hathaway probably m...,,Warren Buffett's Berkshire Hathaway likely sco...
...,...,...,...,...,...,...
2177,2021-01-05 14:42 UTC,https://www.wtap.com/2021/01/05/trump-says-hel...,"[Lisa Mascaro, Mary Clare Jalonick, Kevin Frek...",WASHINGTON (AP) — Republicans mounting an unpr...,,"Dividing party, Republicans poised to challeng..."
2178,2021-01-05 14:42 UTC,https://www.digitalspy.com/tv/reality-tv/a3512...,[Rianne Houghton],The Masked Singer UK has already revealed the ...,,Stacey Solomon and Joe Swash respond to Masked...
2179,2021-01-05 14:40 UTC,https://www.digitaltrends.com/home/how-to-conn...,"[Michael Archambault, Tyler Lacoma, January]",Digital Trends may earn a commission when you ...,,How to connect your Alexa-enabled device to Wi-Fi
2180,2021-01-05 22:00 UTC,https://www.investors.com/market-trend/stock-m...,"[Investor'S Business Daily, Scott Lehtonen]",Dow Jones futures and S&P 500 futures were low...,,"Dow Jones Futures Fall, While Tech Futures Sel..."


In [196]:
v

,time,source,author,fulltext,summary,title
0,2021-01-05 22:18 UTC,https://www.cnbc.com/2021/01/05/apple-proxy-20...,[Kif Leswing],"Tim Cook, chief executive officer of Apple, sp...",,Apple adds new section about antitrust risk to...
1,2021-01-06 00:04 UTC,https://www.malaymail.com/news/money/2021/01/0...,[],Living up to Apple’s stated values regarding s...,,"Apple ties exec bonuses to social, environment..."
2,2021-01-05 23:35 UTC,https://www.ibtimes.com/apple-ties-exec-bonuse...,[],Apple is making environmental and social value...,,"Apple Ties Exec Bonuses To Social, Environment..."
3,2021-01-05 23:24 UTC,https://www.reuters.com/article/apple-compensa...,[Stephen Nellis],SAN FRANCISCO (Reuters) -Apple Inc said in its...,,Apple will modify executive bonuses based on e...
4,2021-01-05 23:24 UTC,https://uk.reuters.com/article/apple-compensat...,[Stephen Nellis],SAN FRANCISCO (Reuters) -Apple Inc said in its...,,Apple will modify executive bonuses based on e...
...,...,...,...,...,...,...
2177,2021-01-05 14:42 UTC,https://www.wtap.com/2021/01/05/trump-says-hel...,"[Lisa Mascaro, Mary Clare Jalonick, Kevin Frek...",WASHINGTON (AP) — Republicans mounting an unpr...,,"Dividing party, Republicans poised to challeng..."
2178,2021-01-05 14:42 UTC,https://www.digitalspy.com/tv/reality-tv/a3512...,[Rianne Houghton],The Masked Singer UK has already revealed the ...,,Stacey Solomon and Joe Swash respond to Masked...
2179,2021-01-05 14:40 UTC,https://www.digitaltrends.com/home/how-to-conn...,"[Michael Archambault, Tyler Lacoma, January]",Digital Trends may earn a commission when you ...,,How to connect your Alexa-enabled device to Wi-Fi
2180,2021-01-05 22:00 UTC,https://www.investors.com/market-trend/stock-m...,"[Investor'S Business Daily, Scott Lehtonen]",Dow Jones futures and S&P 500 futures were low...,,"Dow Jones Futures Fall, While Tech Futures Sel..."


In [158]:
test_list[1800]

('https://abc7news.com/localish/medical-oddities-marvels-find-a-home-at-the-mutter-museum-/8980399/',
 '2021-01-05 18:16 UTC')

In [169]:
l = [record for record in raw_output if record]

In [187]:
pd.DataFrame(l[0:1])

,time,source,author,fulltext,summary,title
0,2021-01-05 22:18 UTC,https://www.cnbc.com/2021/01/05/apple-proxy-20...,[Kif Leswing],"Tim Cook, chief executive officer of Apple, sp...",,Apple adds new section about antitrust risk to...


In [188]:
l[2]

'Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/ewanspence/2021/01/05/apple-macbook-pro-m1-m1x-macos-big-arm-leak-specs-release-date/ on URL https://www.forbes.com/sites/ewanspence/2021/01/05/apple-macbook-pro-m1-m1x-macos-big-arm-leak-specs-release-date/'